In [1]:
from WindPy import w
from datetime import datetime, timedelta
w.start()
date = '2022-11-18'
code = ['005754.OF', '005756.OF', '008911.OF', '008913.OF', '007935.OF', '007936.OF', '008696.OF', '004827.OF', '006851.OF']
w.wsd(code[0], "sec_name,nav,NAV_adj_return1,return_1m,return_3m,return_1y", '2022-11-5', '2022-11-18', "annualized=1", usedf=True)[1]

Welcome to use Wind Quant API for Python (WindPy)!

COPYRIGHT (C) 2020 WIND INFORMATION CO., LTD. ALL RIGHTS RESERVED.
IN NO CIRCUMSTANCE SHALL WIND BE RESPONSIBLE FOR ANY DAMAGES OR LOSSES CAUSED BY USING WIND QUANT API FOR Python.


,SEC_NAME,NAV,NAV_ADJ_RETURN1,RETURN_1M,RETURN_3M,RETURN_1Y
2022-11-07,平安短债A,1.1488,0.008705,2.492929,1.922233,3.170184
2022-11-08,平安短债A,1.1486,-0.017409,2.283034,1.710601,3.124439
2022-11-09,平安短债A,1.1485,-0.008706,2.178233,1.710751,3.124719
2022-11-10,平安短债A,1.1482,-0.026121,1.031175,1.570145,3.070018
2022-11-11,平安短债A,1.1478,-0.034837,0.514333,1.500236,3.034111
2022-11-14,平安短债A,1.1463,-0.130685,-1.325670,0.974999,2.890225
2022-11-15,平安短债A,1.1453,-0.087237,-2.334451,0.416772,2.791240
2022-11-16,平安短债A,1.1443,-0.087313,-3.333787,-0.034663,2.683058
2022-11-17,平安短债A,1.1436,-0.061173,-4.420734,-0.277055,2.601830
2022-11-18,平安短债A,1.1438,0.017489,-4.321913,-0.242434,2.601363


In [3]:
from WindPy import w
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
try:
    from typing import Literal
except ImportError:
    from typing_extensions import Literal
from dateutil.relativedelta import relativedelta

In [45]:

class Wind_Exporter:
    """
    Wind_Exporter
    =====
    code: list
        股票代码, 例如['000001.SZ', '000002.SZ']
    indicator: str
        指标, 例如'close'
    method: str (default: 'wsd')
        wsd: 单日期 
        wss: 日期截面数据
    Date_List: list (default: None)
        日期列表, 例如['2020-01-01', '2020-01-02']
    StartDate: str (default: None)
        起始日期, 例如'2020-01-01'
    EndDate: str (default: None)
        结束日期, 例如'2020-01-02'
    options: str (default: None)
        选项, 遵循Wind API的options参数
    """

    def __init__(self, code=None, indicator: str = None, method: Literal['wsd', 'wss'] = 'wsd', 
                    Date_List: list = None, StartDate: Literal['before1m', 'before1y', None] = None, EndDate=None, options=None):
        self.method = method
        self.code = code if isinstance(code, list) else [
            i for i in code.split(',')]
        self.EndDate = EndDate if EndDate else datetime.today().strftime('%Y-%m-%d')
        self.Date_List = Date_List
        self.StartDate = EndDate if StartDate is None else StartDate
        self.set_date()
        self.indicator = indicator
        self.options = options
        self.data = []

    def __len__(self):
        return len(self.data)

    def __repr__(self):
        return f'Wind_Exporter({self.code}, {self.indicator}, {self.method}, {self.Date_List}, {self.StartDate}, {self.EndDate}, {self.options}), With data length {len(self)}'

    def check_connection(func):
        if not w.isconnected():
            w.start()
        return func

    @staticmethod
    def create_date_col(df, date):
        df['date'] = date
        column = np.roll(np.arange(len(df.columns)), 1)
        column[0], column[1] = column[1], column[0]
        return df.iloc[:, column]
    

#my_dict = {'code':['005754.OF', '005756.OF', '008911.OF', '008913.OF', '007935.OF', '007936.OF', '008696.OF', '004827.OF', '006851.OF'],
            # 'indicator':"sec_name,nav_date,nav,NAV_adj_return1,return_1m,return_3m,return_1y", 'options':"annualized=1", 'method':"wsd"}
    @classmethod
    def from_dict(cls, data: dict):
        return cls(**data) 
        

    def set_date(self):
        date_before_1m = (datetime.strptime(self.EndDate, '%Y-%m-%d') -
                          relativedelta(months=1) + relativedelta(days=1)).strftime('%Y%m%d')
        date_before_1y = (datetime.strptime(self.EndDate, '%Y-%m-%d') -
                          relativedelta(years=1) + relativedelta(days=1)).strftime('%Y%m%d')
        if self.StartDate == 'before1m':
            self.StartDate = date_before_1m
        elif self.StartDate == 'before1y':
            self.StartDate = date_before_1y
        else:
            pass

    @check_connection
    def get_data(self, output: Literal['df', 'excel'] = None, round_=None):
        if self.method == 'wsd':
            self.get_data_wsd()
        elif self.method == 'wss':
            self.get_data_wss()
        else:
            raise ValueError('method must be wsd or wss')

        if round_ is not None:
            for i in range(len(self.data)):
                self.data[i] = self.data[i].round(round_)
        if output == 'df':
            return self.data
        elif output == 'excel':
            pass
        else:
            return self

    def get_data_wsd(self):
        if self.Date_List is None:
            if self.StartDate == self.EndDate:
                multi_data = []
                for i in self.code:
                    data_ = w.wsd(i, self.indicator, self.StartDate,
                                  self.EndDate, options=self.options, usedf=True)
                    multi_data.append(data_[1])
                self.data.append(pd.concat(multi_data, axis=0))
            else:
                multi_data = []
                for i in self.code:
                    data_ = w.wsd(i, self.indicator, self.StartDate,
                                  self.EndDate, options=self.options, usedf=True)
                    multi_data.append(data_[1])
                self.data.append(multi_data)
        else:
            for date in self.Date_List:
                multi_data = []
                for i in self.code:
                    data_ = w.wsd(i, self.indicator, date, date,
                                  options=self.options, usedf=True)
                    multi_data.append(data_[1])
                self.data.append(pd.concat(multi_data, axis=0))
        return self

# w.wss("002450.OF","NAV_adj_return","startDate=20221117;endDate=20221118")
# w.wss("009878.OF", "sec_name","startDate=20221117;endDate=20221118")

    @check_connection
    def combine_wss(self, group_data: list):
        for i in group_data:
            data_ = []
            for j in i:
                data_.append(j[1])
            data_ = pd.concat(data_, axis=1)
            self.data.append(data_)
        return self

    def get_data_wss(self):
        self.StartDate = self.StartDate.replace('-', '')
        self.EndDate = self.EndDate.replace('-', '')
        args = [self.options, self.StartDate, self.EndDate]
        if self.options is None:
            args = args[1:]
        options = f'startDate={args[0]};endDate={args[1]};tradeDate={args[1]}' if len(
            args) == 2 else f'{args[0]};startDate={args[1]};endDate={args[2]};tradeDate={args[2]}'
        print(options)
        data_ = w.wss(self.code, self.indicator, options=options, usedf=True)
        self.data.append(data_[1])
        return self

    def add_data(self, we_obj, method: Literal['concat', 'append'] = 'concat', round_=None):
        if method == 'concat':
            we_obj.get_data(round_=round_)
            self.data[-1] = pd.concat([self.data[-1], we_obj.data[-1]], axis=1)
        elif method == 'append':
            we_obj.get_data(round_=round_)
            for i in we_obj.data:
                self.data.append(i)
        else:
            raise ValueError('method must be concat or append')
        return self

    def excel_export(self, path=None, sheet_name: list = None, column_name: list = None):
        """
        导出为Excel
        =====
        path: str (default: None)
            导出路径, 例如'./data.xlsx', 默认为None, 会在output目录下生成{日期/开始日期_结束日期}.xlsx
        sheet_name: list (default: None)
            sheet名称, 例如['sheet1', 'sheet2'], 默认为None, 会使用工作簿数量作为sheet名称
        column_name: list (default: None)
            列名称, 二维数组, 例如[['col1', 'col2'], ['col1', 'col2']], 默认为None, 会使用Wind默认列名称
        """
        if path is None:
            if self.Date_List is None:
                if self.StartDate == self.EndDate:
                    path = f'./output/{self.EndDate}.xlsx'
                else:
                    path = f'./output/{self.StartDate}_{self.EndDate}.xlsx'
            else:
                path = f'./output/{self.Date_List[0]}.xlsx'
        if sheet_name is None:
            sheet_name = np.arange(len(self.data)) + 1
        if column_name is not None:
            try:
                _ = column_name[0][0]
            except:
                raise ValueError('column_name 应当是二维列表')
            for i, data in enumerate(self.data):
                try:
                    data.columns = column_name[i]
                    print(column_name[i])
                    print('更名完成')
                except:
                    raise ValueError('列名数与数据不匹配')
        with pd.ExcelWriter(path, datetime_format='YYYY-MM-DD') as writer:
            for i in enumerate(self.data):
                i[1].to_excel(writer, sheet_name=sheet_name[i[0]])
        return self

In [38]:
code_for_zhai = ['005754.OF', '005756.OF', '008911.OF', '008913.OF', '007935.OF', '007936.OF', '008696.OF', '004827.OF', '006851.OF']
code_for_ETF = ['516760.OF', '516820.OF', '515700.OF', '561600.OF']
code_cundan_str = """015645.OF,015644.OF,015826.OF,015862.OF,014437.OF,015823.OF,015648.OF,014427.OF,015822.OF,015647.OF,015875.OF,
                        014426.OF,015864.OF,015861.OF,015646.OF,015643.OF,014430.OF,015956.OF,015825.OF,014428.OF,015827.OF,015863.OF,014429.OF,015944.OF,015955.OF,016082.OF,016063.OF,016083.OF"""
code_for_meiri_str = """002450.OF,004827.OF,015645.OF,008694.OF,005754.OF,700003.OF,000739.OF,007935.OF,009661.OF,009878.OF,
                        010126.OF,014460.OF,013767.OF,013687.OF,004390.OF,012475.OF,007893.OF,885001.WI"""

choice_data = "2022-11-24"
a = Wind_Exporter(code=code_for_zhai, indicator="sec_name,nav_date,nav,NAV_adj_return1,return_1m,return_3m,return_1y",options="annualized=1",method="wsd",EndDate=choice_data)
b = Wind_Exporter(code=code_for_ETF, indicator="sec_name,nav_date,nav,NAV_adj_return1,return_1w,return_1m",options="annualized=1",method="wsd",EndDate=choice_data)
c = Wind_Exporter(code="015645.OF", indicator="sec_name,nav_date,nav,NAV_adj_return1,return_1w,return_1m",options="annualized=1",method="wsd",EndDate=choice_data)
d = Wind_Exporter(code=code_cundan_str, indicator="sec_name,nav_date,return,risk_annualintervalyield,issue_date,fund_setupdate",options="annualized=0",method="wss",EndDate=choice_data)
e = Wind_Exporter(code=code_for_meiri_str, indicator="sec_name,nav_date,nav,NAV_adj_return1,NAV_adj_return,return_ytd",options="annualized=0",method="wss",StartDate="before1m",EndDate=choice_data)
e_ = Wind_Exporter(code=code_for_meiri_str, indicator="NAV_adj_return",method="wss",StartDate="before1y",EndDate=choice_data)
a.get_data().add_data(b, method='append').add_data(c, method='append').add_data(d, method='append').add_data(e, method='append').add_data(e_, method='concat')


annualized=0;startDate=20221124;endDate=20221124;tradeDate=20221124
annualized=0;startDate=20221025;endDate=20221124;tradeDate=20221124
startDate=20211125;endDate=20221124;tradeDate=20221124


Wind_Exporter(['005754.OF', '005756.OF', '008911.OF', '008913.OF', '007935.OF', '007936.OF', '008696.OF', '004827.OF', '006851.OF'], sec_name,nav_date,nav,NAV_adj_return1,return_1m,return_3m,return_1y, wsd, None, 2022-11-24, 2022-11-24, annualized=1), With data length 5

In [51]:
a.data[3]

,SEC_NAME,NAV_DATE,RETURN,RISK_ANNUALINTERVALYIELD,ISSUE_DATE,FUND_SETUPDATE
015645.OF,平安同业存单指数7天持有,2022-11-24,0.029706,11.450513,2022-05-05,2022-05-18
015644.OF,华夏同业存单指数7天持有,2022-11-24,0.029735,11.462488,2022-04-25,2022-05-10
015826.OF,广发同业存单指数7天持有,2022-11-24,0.019818,7.500720,2022-05-23,2022-06-02
015862.OF,景顺长城同业存单指数7天持有,2022-11-24,0.039588,15.542545,2022-05-30,2022-06-06
014437.OF,鹏华同业存单指数7天持有,2022-11-24,0.039059,15.319502,2021-12-03,2021-12-13
015823.OF,银华同业存单指数7天持有,2022-11-24,0.019814,7.499179,2022-05-23,2022-06-08
015648.OF,兴银同业存单指数7天持有,2022-11-24,0.039671,15.577309,2022-04-26,2022-05-11
014427.OF,富国同业存单指数7天持有,2022-11-24,0.039089,15.332351,2021-12-03,2021-12-15
015822.OF,易方达同业存单指数7天持有,2022-11-24,0.049544,19.816536,2022-05-26,2022-06-02
015647.OF,淳厚同业存单指数7天持有,2022-11-24,0.039651,15.569018,2022-05-05,2022-05-20


In [52]:
a.excel_export(sheet_name=['债', 'ETF', '同业', '存单', '每日'], column_name=[['证券简称', '基金净值日期','单位净值', '当期复权单位净值增长率', '近1月回报', '近3月回报', '近1年回报'],
                ['证券简称', '基金净值日期','单位净值', '当期复权单位净值增长率', '近1周回报', '近1月回报'],['证券简称', '基金净值日期','单位净值', '当期复权单位净值增长率', '近1周回报', '近1月回报'],
                ['证券简称', '基金净值日期','区间回报', '区间收益率', '发行日期', '基金成立日'], ['证券简称', '基金净值日期','单位净值', '当期复权单位净值增长率', '复权单位净值增长率(截止日1月前)', '今年以来回报', '复权单位净值增长率(截止日1年前)']],
                path='./output/{}.xlsx'.format(choice_data))

Wind_Exporter(['005754.OF', '005756.OF', '008911.OF', '008913.OF', '007935.OF', '007936.OF', '008696.OF', '004827.OF', '006851.OF'], sec_name,nav_date,nav,NAV_adj_return1,return_1m,return_3m,return_1y, wsd, None, 2022-11-24, 2022-11-24, annualized=1), With data length 5

In [32]:
# code = [[w.wss("516760.OF,516820.OF,515700.OF,561600.OF", "swing","tradeDate=20221122;cycle=D", usedf=True)],
# [w.wss("002450.OF","NAV_adj_return","startDate=20221117;endDate=20221118", usedf=True),w.wss("009878.OF", "sec_name","startDate=20221117;endDate=20221118", usedf=True)]]
code = ['002450.OF','009878.OF']
code_for_meiri_str = """002450.OF,004827.OF,015645.OF,008694.OF,005754.OF,700003.OF,000739.OF,007935.OF,009661.OF,009878.OF,010126.OF,014460.OF,013767.OF,013687.OF,004390.OF,012475.OF,007893.OF,885001.WI"""
a = Wind_Exporter(method='wss', code=code_for_meiri_str, indicator="sec_name,nav_date,nav,NAV_adj_return1,NAV_adj_return,return_ytd",StartDate='before1m',EndDate='2022-11-24',options="annualized=0")
b = Wind_Exporter(method='wss', code=code_for_meiri_str, indicator="NAV_adj_return",StartDate='before1y',EndDate='2022-11-24')
a.get_data(round_=4)

annualized=0;startDate=20221025;endDate=20221124;tradeDate=20221124


Wind_Exporter(['002450.OF', '004827.OF', '015645.OF', '008694.OF', '005754.OF', '700003.OF', '000739.OF', '007935.OF', '009661.OF', '009878.OF', '010126.OF', '014460.OF', '013767.OF', '013687.OF', '004390.OF', '012475.OF', '007893.OF', '885001.WI'], sec_name,nav_date,nav,NAV_adj_return1,NAV_adj_return,return_ytd, wss, None, 20221025, 20221124, annualized=0), With data length 1

In [33]:
a.add_data(b, method='concat',round_=4)


startDate=20211125;endDate=20221124;tradeDate=20221124


Wind_Exporter(['002450.OF', '004827.OF', '015645.OF', '008694.OF', '005754.OF', '700003.OF', '000739.OF', '007935.OF', '009661.OF', '009878.OF', '010126.OF', '014460.OF', '013767.OF', '013687.OF', '004390.OF', '012475.OF', '007893.OF', '885001.WI'], sec_name,nav_date,nav,NAV_adj_return1,NAV_adj_return,return_ytd, wss, None, 20221025, 20221124, annualized=0), With data length 1

In [34]:
a.data[0]

,SEC_NAME,NAV_DATE,NAV,NAV_ADJ_RETURN1,NAV_ADJ_RETURN,RETURN_YTD,NAV_ADJ_RETURN
002450.OF,平安睿享文娱A,2022-11-24,2.1950,-0.3631,-3.6858,-14.3247,-14.2913
004827.OF,平安中短债A,2022-11-24,1.1447,0.1312,-0.8746,1.9323,2.4157
015645.OF,平安同业存单指数7天持有,2022-11-24,1.0102,0.0297,0.0495,1.0200,NaN
008694.OF,平安元盛超短债A,2022-11-24,1.0708,0.1028,-0.1771,1.3727,1.6421
005754.OF,平安短债A,2022-11-24,1.1452,0.0787,-0.2786,2.3048,2.6625
700003.OF,平安策略先锋,2022-11-24,5.4440,-0.7656,-5.8783,-16.6565,-19.0242
000739.OF,平安新鑫先锋A,2022-11-24,2.7320,0.1466,3.4065,-11.4137,-14.7848
007935.OF,平安惠澜纯债A,2022-11-24,1.1289,0.1508,-0.6425,2.4038,3.1807
009661.OF,平安研究睿选A,2022-11-24,0.8554,0.0936,4.7386,-27.4593,-27.9906
009878.OF,平安低碳经济A,2022-11-24,0.9181,-0.5309,3.4479,-20.9080,-21.9635


In [17]:
indicator = "sec_name,nav,NAV_adj_return1,return_1m,return_3m,return_1y"
date = ['2022-11-18', '2022-11-17']
code = ['005754.OF', '005756.OF', '008911.OF', '008913.OF', '007935.OF', '007936.OF', '008696.OF', '004827.OF', '006851.OF']
Wind_Exporter(code, method='wsd', indicator=indicator, StartDate='before1y', options="annualized=1")

Wind_Exporter(['005754.OF', '005756.OF', '008911.OF', '008913.OF', '007935.OF', '007936.OF', '008696.OF', '004827.OF', '006851.OF'], sec_name,nav,NAV_adj_return1,return_1m,return_3m,return_1y, wsd, None, 20211124, 2022-11-23, annualized=1)

In [112]:
code_for_meiri_str = """002450.OF,004827.OF,015645.OF,008694.OF,005754.OF,700003.OF,000739.OF,007935.OF,009661.OF,009878.OF,010126.OF,014460.OF,013767.OF,013687.OF,004390.OF,012475.OF,007893.OF,885001.WI"""
w.wss(code_for_meiri_str, "sec_name,nav,NAV_adj_return1,NAV_adj_return,return_ytd","tradeDate='20221118';startDate='20221018';endDate='20221118';annualized=0")
# w.wss("002450.OF","NAV_adj_return","startDate=20221117;endDate=20221118")
# w.wss("009878.OF", "sec_name","startDate=20221117;endDate=20221118")
# w.wsd("015645.OF", "sec_name,nav,NAV_adj_return1,return_1w,return_1m", date, date, "annualized=1")

.ErrorCode=-40521008
.Codes=[ErrorReport]
.Fields=[OUTMESSAGE]
.Times=[20221123 23:01:28]
.Data=[[CWSSServiceresponse error.]]